In [1]:
import time
import pandas as pd
import numpy as np
import typing as tp

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
iris = load_iris()

In [3]:
X = iris.data[(iris.target == 1) | (iris.target == 2)]
y = (iris.target[(iris.target == 1) | (iris.target == 2)] == 2).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [5]:
class MyLogisticRegression:
    """Классификатор логистической регрессии с настраиваемыми методами оптимизации.
    Args:
    max_iter : int, default=1000
        Максимальное количество итераций для оптимизации.
    learning_rate : float, default=0.01
        Скорость обучения для оптимизации градиентного спуска.
    optimizer : str, default='gd'
        Метод оптимизации.
        Доступные варианты:
        - 'gd' (градиентный спуск),
        - 'rmsprop' (RMSProp),
        - 'nadam' (Нестеров-Адам).
    momentum : float, default=0.9
        Параметр момента для оптимизации по моменту Нестерова.
    beta : float, default=0.9
        Параметр бета для оптимизации RMSProp.
    epsilon : float, default=1e-8
        Параметр эпсилон, чтобы избежать деления на ноль при оптимизации RMSProp.
    threshold : float, default=0.5
        Порог для предсказания класса. Значения выше порога
        предсказываются как 1, в противном случае - как 0.
    random_state : int, default=None
        Случайная затравка для воспроизводимости.
    """
    def __init__(self,
                 max_iter: int = 1000,
                 learning_rate: float = 0.01,
                 optimizer: str = 'gd',
                 momentum: float = 0.9,
                 beta: float = 0.9,
                 epsilon: float = 1e-8,
                 threshold: float = 0.5,
                 random_state: int = None) -> None:
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.random_state = random_state
        self.momentum = momentum
        self.beta = beta
        self.epsilon = epsilon
        self.threshold = threshold
        self.weights = None
    
    def _sigmoid(self, z: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-z))
    
    def _compute_gradient(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        m = X.shape[0]
        y_pred = self._sigmoid(np.dot(X, self.weights))
        gradient = np.dot(X.T, (y_pred - y)) / m
        return gradient

    def _gradient_descent(self, X: np.ndarray, y: np.ndarray) -> None:
        for _ in range(self.max_iter):
            gradient = self._compute_gradient(X, y)
            self.weights -= self.learning_rate * gradient

    def _rmsprop(self, X: np.ndarray, y: np.ndarray) -> None:
        E_grad_squared = np.zeros_like(self.weights)
        for _ in range(self.max_iter):
            gradient = self._compute_gradient(X, y)
            E_grad_squared = self.beta * E_grad_squared + (1 - self.beta) * np.square(gradient)
            self.weights -= (self.learning_rate / np.sqrt(E_grad_squared + self.epsilon)) * gradient

    def _nesterov_mometum(self, X: np.ndarray, y: np.ndarray) -> None:
        velocity = np.zeros_like(self.weights)
        for _ in range(self.max_iter):
            gradient = self._compute_gradient(X, y)
            velocity = self.momentum * velocity + self.learning_rate * gradient
            self.weights -= velocity

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        num_features = X.shape[1]
        self.weights = np.zeros(num_features)
        
        if self.optimizer == 'gd':
            self._gradient_descent(X, y)
        elif self.optimizer == 'rmsprop':
            self._rmsprop(X, y)
        elif self.optimizer == 'nadam':
            self._nesterov_mometum(X, y)
        else:
            raise ValueError('Unknown optimizer. Please, select "rmsprop", "gd" or "nadam"')
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        y_pred = self._sigmoid(np.dot(X, self.weights))
        return (y_pred >= self.threshold).astype(int)

In [6]:
model = MyLogisticRegression(optimizer='nadam')
model.fit(X_train, y_train)

In [7]:
predictions = model.predict(X_test)

In [8]:
accuracy_score(y_test, predictions)

1.0

In [9]:
def train_and_evaluate(optimizer: str) -> tp.Dict:
    """Функция для создания таблицы с результатами работы логистической регресси
    и разными методами оптимизации. 

    Args:
        optimizer (str): метод оптимизации

    Returns:
        tp.Dict: словарь со следующими ключами: метод, метрика и время работы.
    """
    start_time = time.time()
    model = MyLogisticRegression(optimizer=optimizer)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    end_time = time.time()
    execution_time = end_time - start_time
    return {'Метод': optimizer, 'Метрика': accuracy, 'Время работы': execution_time}


optimizers = ['gd', 'rmsprop', 'nadam']

results = []
for optimizer in optimizers:
    result = train_and_evaluate(optimizer)
    results.append(result)

df = pd.DataFrame(results)

In [10]:
df.head()

,Метод,Метрика,Время работы
0,gd,0.95,0.009771
1,rmsprop,1.00,0.007861
2,nadam,1.00,0.006759
